In [1]:
import glob
from pathlib import Path
from datetime import datetime

# Simula tu función get_my_path("data_raw")
# Cambia esta ruta a la REAL de tu proyecto si es diferente
DATA_RAW_ROOT = Path("/home/legion/bulk/MAIE_tesis2026/f01_code/MAIE_tesis_github/data_raw/goes_raw")

print(f"DATA_RAW_ROOT: {DATA_RAW_ROOT}")
print(f"Existe? {DATA_RAW_ROOT.exists()}")
print(f"Subcarpetas: {list(DATA_RAW_ROOT.iterdir())}")

DATA_RAW_ROOT: /home/legion/bulk/MAIE_tesis2026/f01_code/MAIE_tesis_github/data_raw/goes_raw
Existe? True
Subcarpetas: [PosixPath('/home/legion/bulk/MAIE_tesis2026/f01_code/MAIE_tesis_github/data_raw/goes_raw/noaa-goes19')]


In [2]:
# Simula un slot de hora 00 para LSTF (como en tu plan)
bucket = "noaa-goes19"
product_id = "ABI-L2-LSTF"
year = "2026"
day = "003"
h_str = "00"
sat_id = "19"

# Patrón original (el que no funcionó)
original_regex = f"{bucket}/{product_id}/{year}/{day}/{h_str}/OR_ABI-L2-LSTF-M6_G{sat_id}*s{year}{day}{h_str}00*.nc"
print(f"Patrón original: {original_regex}")

# Patrón corregido: solo hasta la hora, sin minutos exactos
corrected_regex = f"{bucket}/{product_id}/{year}/{day}/{h_str}/OR_ABI-L2-LSTF-M6_G{sat_id}_s{year}{day}{h_str}*.nc"
print(f"Patrón corregido: {corrected_regex}")

# Buscar con glob desde data_raw/goes_raw
full_pattern_original = DATA_RAW_ROOT / original_regex
full_pattern_corrected = DATA_RAW_ROOT / corrected_regex

print("\nBuscando con patrón original...")
candidates_original = glob.glob(str(full_pattern_original))
print(f"Encontrados con original: {len(candidates_original)}")
if candidates_original:
    print("Primeros 3:")
    for p in candidates_original[:3]:
        print(p)

print("\nBuscando con patrón corregido...")
candidates_corrected = glob.glob(str(full_pattern_corrected))
print(f"Encontrados con corregido: {len(candidates_corrected)}")
if candidates_corrected:
    print("Primeros 3:")
    for p in candidates_corrected[:3]:
        print(p)

# Si hay matches, mostrar el más reciente
if candidates_corrected:
    most_recent = max([Path(p) for p in candidates_corrected], key=lambda p: p.stat().st_mtime)
    print(f"\nArchivo más reciente encontrado: {most_recent}")
    print(f"Modificado: {datetime.fromtimestamp(most_recent.stat().st_mtime).strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Tamaño: {most_recent.stat().st_size / (1024*1024):.2f} MB")

Patrón original: noaa-goes19/ABI-L2-LSTF/2026/003/00/OR_ABI-L2-LSTF-M6_G19*s20260030000*.nc
Patrón corregido: noaa-goes19/ABI-L2-LSTF/2026/003/00/OR_ABI-L2-LSTF-M6_G19_s202600300*.nc

Buscando con patrón original...
Encontrados con original: 0

Buscando con patrón corregido...
Encontrados con corregido: 0


In [3]:
# Simula todos los slots de hora 00 (para LSTF solo hay 1 por hora)
hour = "00"
corrected_regex_hour = f"{bucket}/{product_id}/{year}/{day}/{hour}/OR_ABI-L2-LSTF-M6_G{sat_id}_s{year}{day}{hour}*.nc"
full_pattern_hour = DATA_RAW_ROOT / corrected_regex_hour

print(f"Buscando en hora {hour}: {full_pattern_hour}")
candidates_hour = glob.glob(str(full_pattern_hour))
print(f"Encontrados en hora {hour}: {len(candidates_hour)}")

if candidates_hour:
    print("Archivos encontrados:")
    for p in sorted(candidates_hour):
        print(p)

Buscando en hora 00: /home/legion/bulk/MAIE_tesis2026/f01_code/MAIE_tesis_github/data_raw/goes_raw/noaa-goes19/ABI-L2-LSTF/2026/003/00/OR_ABI-L2-LSTF-M6_G19_s202600300*.nc
Encontrados en hora 00: 0


In [ ]:
import boto3
import os
from botocore import UNSIGNED
from botocore.config import Config

# --- CONFIGURACIÓN DE PRUEBA ---
bucket_name = 'noaa-goes19'
# LSTF es Full Disk, 2026 es el año, 003 es el día juliano
prefix = 'ABI-L2-LSTF/2026/003/' 
#prefix = 'noaa-goes19/ABI-L2-LSTF/2026/003/OR_ABI-L2-LSTF-M6_G19_s202600300*' 
prefix = 'ABI-L2-LSTF/2026/003/OR_ABI-L2-LSTF-M6_G19_s202600300*' 

str_offd = './pruebas_goes'

# Crear carpeta si no existe
if not os.path.exists(str_offd):
    os.makedirs(str_offd)

# Inicializar cliente S3 sin credenciales (acceso público)
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

print(f"Buscando archivos en s3://{bucket_name}/{prefix}...")

# Listar los archivos disponibles en esa ruta
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
response


Buscando archivos en s3://noaa-goes19/ABI-L2-LSTF/2026/003/OR_ABI-L2-LSTF-M6_G19_s20260030000228_e20260030009536_c20260030015142.nc...


{'ResponseMetadata': {'RequestId': '3ZMNHQGME8GE1PWS',
  'HostId': '0HAaDka7AWRVJitn954ay0dlJ1H8kFvfErwjs7hcs/HFRQ15ym1QiYpk4mMsyhFry+c/fDEb8IE72dt+17w6RZsZGb4r+q9G',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '0HAaDka7AWRVJitn954ay0dlJ1H8kFvfErwjs7hcs/HFRQ15ym1QiYpk4mMsyhFry+c/fDEb8IE72dt+17w6RZsZGb4r+q9G',
   'x-amz-request-id': '3ZMNHQGME8GE1PWS',
   'date': 'Fri, 27 Feb 2026 21:26:13 GMT',
   'x-amz-bucket-region': 'us-east-1',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Name': 'noaa-goes19',
 'Prefix': 'ABI-L2-LSTF/2026/003/OR_ABI-L2-LSTF-M6_G19_s20260030000228_e20260030009536_c20260030015142.nc',
 'MaxKeys': 1000,
 'EncodingType': 'url',
 'KeyCount': 0}

In [ ]:
if 'Contents' in response:
    # Tomamos el primer archivo de la lista para la prueba
    first_file = response['Contents'][0]
    file_key = first_file['Key']
    file_name = os.path.basename(file_key)
    
    print(f"Archivo encontrado: {file_name}")
    print(f"Descargando...")
    
    path_final = os.path.join(str_offd, file_name)
    s3.download_file(bucket_name, file_key, path_final)
    
    print(f"¡Éxito! Archivo descargado en: {path_final}")
else:
    print("No se encontraron archivos. Revisa si el satélite o la fecha son correctos.")